# Tree Reconstruction from Video

To install before first use:
 1) Download and place the Agisoft Metashape 2.02 WHS file from https://www.agisoft.com/downloads/installer/ into the `bin` folder.
 2) Download and place the latest release of IRSS Media Tools from https://github.com/lukasdragon/MediaTools/releases into the `exec` folder.
 3) run the following command in the terminal: ```python setup.py```


In [1]:
import shutil
from pathlib import Path

import IRSSMediaTools
from helpers import get_all_files
import Metashape

In [2]:
DATA_PATH = Path("data")
VIDEO_PATH = DATA_PATH / "video"
TEMP_PATH = Path("tmp")

In [3]:
if TEMP_PATH.exists():
    shutil.rmtree(TEMP_PATH)

In [4]:
video_files = get_all_files(VIDEO_PATH, "*")

for file in video_files:
    print(file)

data\video\DJI_0152.mp4


In [5]:
for video_file in video_files:
    IRSSMediaTools.extract_frames(video_file, TEMP_PATH / "frames", 0.75)

Extracting frames from video at path: data\video\DJI_0152.mp4
E:/lukas_olson_msc/Dataspell/Reconstruction/data/video/DJI_0152.mp4

# of frames: 12654, Frame Size: {Width=1920, Height=1080}, is Variable FR: 

False, Avg. FR: 59.94005994005994

Rotation: 0, is Interlaced: False, Pixel Format yuv420p, Duration: 

00:03:31.1109000, Codec: hevc

Metadata: creation_time: 2021-05-20T13:04:28.000000Z; language: und; handler_name: VideoHandler; vendor_id: [0][0][0][0]; Extracting 3164 frames from 

E:\lukas_olson_msc\Dataspell\Reconstruction\data\video\DJI_0152.mp4 to 

E:\lukas_olson_msc\Dataspell\Reconstruction\tmp\frames.

Extracting Frames: 0%

Extracting Frames: 25%

Extracting Frames: 50%

Extracting Frames: 75%

Extracting Frames: 95%

Extracting Frames: 96%

Extracting Frames: 97%

Extracting Frames: 99%

Finished extracting 3164 frames in 79.22 seconds.



In [6]:
# Create Metashape project
doc = Metashape.Document()

# Create new chunk
chunk = doc.addChunk()

for video_file in video_files:
    # Get all frame paths for current video file
    frame_paths = get_all_files(TEMP_PATH / "frames", "*")

    # Add frames to chunk
    chunk.addPhotos([str(path) for path in frame_paths])

In [ ]:
# Matching photos
chunk.matchPhotos(downscale=1, generic_preselection=True, reference_preselection=False)
chunk.alignCameras()

In [0]:
doc.save(path=str(DATA_PATH / "metashape_project.psx"))

In [12]:

# Build dense cloud
chunk.buildDepthMaps(quality=Metashape.HighQuality)
chunk.buildDenseCloud()

# Build model
chunk.buildModel(surface=Metashape.Arbitrary, interpolation=Metashape.EnabledInterpolation)

# Build texture
chunk.buildTexture(blending=Metashape.MosaicBlending, size=4096)

# Save the project

RuntimeError: Empty file list

In [ ]:
# Export the dense point cloud
output_path = DATA_PATH / "output"
output_path.mkdir(parents=True, exist_ok=True)

for i, chunk in enumerate(doc.chunks):
    dense_cloud_path = output_path / f"dense_cloud_{i}.ply"  # adjust the extension based on the format you want
    chunk.exportPoints(str(dense_cloud_path), source_data=Metashape.DenseCloudData)